In [179]:
# keep this here pls
!pip install xlrd 
#!pip install country_converter

In [200]:
import pandas as pd
import numpy as np
import re

#import warnings
#warnings.filterwarnings('ignore')

#define path to xls
url = 'https://www.sharkattackfile.net/spreadsheets/GSAF5.xls'
df = pd.read_excel(url)
df  # 6992 rows × 23 columns

,Date,Year,Type,Country,State,Location,Activity,Name,Sex,Age,...,Species,Source,pdf,href formula,href,Case Number,Case Number.1,original order,Unnamed: 21,Unnamed: 22
0,2025-01-23 00:00:00,2025.0,Unprovoked,Australia,Western Australia,Shipwreck Cove,Bathing,Boy,M,12,...,Not stated,Bob Myatt GSAF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2025-01-22 00:00:00,2025.0,Unprovoked,Australia,Western Australia,Cheynes beach,Surfing,Dale Kittow,M,37,...,Bronze whaler,Bob Myatt GSAF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2025-01-11 00:00:00,2025.0,Provoked,USA,Hawaii,Off Haleiwa Boat Harbour Oahu,Diving,Male not stated was a dive tour worker,M,23,...,Not stated,Kevin McMurray Trackingsharks.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2025-01-02 00:00:00,2025.0,Unprovoked,New Caledonia,Grande Terre,Islet of Kendek near Koumac,Spearfishing,Robert Cuewapuru,M,40,...,Reportedly Tiger or Bull shark,Johannes Marchand Todd Smith,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2025-01-02 00:00:00,2025.0,Unprovoked,Australia,South Australia,Granites Beach near Westall Streaky Bay,Surfing,Lance Appleby,M,28,...,Great White Shart estimated 4.8m 16 ft,Glen Folkard: Simon De Marchi News.com.au: The...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6987,Before 1903,0.0,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,male,M,NaN,...,NaN,"H. Taunton; N. Bartlett, p. 234",ND-0005-RoebuckBay.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0005,ND.0005,6.0,NaN,NaN
6988,Before 1903,0.0,Unprovoked,AUSTRALIA,Western Australia,NaN,Pearl diving,Ahmun,M,NaN,...,NaN,"H. Taunton; N. Bartlett, pp. 233-234",ND-0004-Ahmun.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0004,ND.0004,5.0,NaN,NaN
6989,1900-1905,0.0,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,Coast Guard personnel,M,NaN,...,NaN,"F. Schwartz, p.23; C. Creswell, GSAF",ND-0003-Ocracoke_1900-1905.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0003,ND.0003,4.0,NaN,NaN
6990,1883-1889,0.0,Unprovoked,PANAMA,NaN,"Panama Bay 8ºN, 79ºW",NaN,Jules Patterson,M,NaN,...,NaN,"The Sun, 10/20/1938",ND-0002-JulesPatterson.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0002,ND.0002,3.0,NaN,NaN


In [125]:
df.head()

,Date,Year,Type,Country,State,Location,Activity,Name,Sex,Age,...,Species,Source,pdf,href formula,href,Case Number,Case Number.1,original order,Unnamed: 21,Unnamed: 22
0,2025-01-23 00:00:00,2025.0,Unprovoked,Australia,Western Australia,Shipwreck Cove,Bathing,Boy,M,12,...,Not stated,Bob Myatt GSAF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2025-01-22 00:00:00,2025.0,Unprovoked,Australia,Western Australia,Cheynes beach,Surfing,Dale Kittow,M,37,...,Bronze whaler,Bob Myatt GSAF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2025-01-11 00:00:00,2025.0,Provoked,USA,Hawaii,Off Haleiwa Boat Harbour Oahu,Diving,Male not stated was a dive tour worker,M,23,...,Not stated,Kevin McMurray Trackingsharks.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2025-01-02 00:00:00,2025.0,Unprovoked,New Caledonia,Grande Terre,Islet of Kendek near Koumac,Spearfishing,Robert Cuewapuru,M,40,...,Reportedly Tiger or Bull shark,Johannes Marchand Todd Smith,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2025-01-02 00:00:00,2025.0,Unprovoked,Australia,South Australia,Granites Beach near Westall Streaky Bay,Surfing,Lance Appleby,M,28,...,Great White Shart estimated 4.8m 16 ft,Glen Folkard: Simon De Marchi News.com.au: The...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [181]:
df.tail()

,Date,Year,Type,Country,State,Location,Activity,Name,Sex,Age,...,Species,Source,pdf,href formula,href,Case Number,Case Number.1,original order,Unnamed: 21,Unnamed: 22
6987,Before 1903,0.0,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,male,M,NaN,...,NaN,"H. Taunton; N. Bartlett, p. 234",ND-0005-RoebuckBay.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0005,ND.0005,6.0,NaN,NaN
6988,Before 1903,0.0,Unprovoked,AUSTRALIA,Western Australia,NaN,Pearl diving,Ahmun,M,NaN,...,NaN,"H. Taunton; N. Bartlett, pp. 233-234",ND-0004-Ahmun.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0004,ND.0004,5.0,NaN,NaN
6989,1900-1905,0.0,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,Coast Guard personnel,M,NaN,...,NaN,"F. Schwartz, p.23; C. Creswell, GSAF",ND-0003-Ocracoke_1900-1905.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0003,ND.0003,4.0,NaN,NaN
6990,1883-1889,0.0,Unprovoked,PANAMA,NaN,"Panama Bay 8ºN, 79ºW",NaN,Jules Patterson,M,NaN,...,NaN,"The Sun, 10/20/1938",ND-0002-JulesPatterson.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0002,ND.0002,3.0,NaN,NaN
6991,1845-1853,0.0,Unprovoked,CEYLON (SRI LANKA),Eastern Province,"Below the English fort, Trincomalee",Swimming,male,M,15,...,NaN,S.W. Baker,ND-0001-Ceylon.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0001,ND.0001,2.0,NaN,NaN


In [201]:
df.shape[0]
df.shape[1]

print("Number of Rows: ", df.shape[0])
print("Number of Columns: ", df.shape[1])

df.columns

# Issues w/ og file –> extra 8 columns on VSC? to preserve NaN
# Date, Year, Type, Country, State, Location, Activity, Name, Sex, Age, Injury, Fatal Y/N, Time, Specie, Source
# check search tool between uppercase and lowercase e.g. Australia / AUSTRALIA

Number of Rows:  6992
Number of Columns:  23


Index(['Date', 'Year', 'Type', 'Country', 'State', 'Location', 'Activity',
       'Name', 'Sex', 'Age', 'Injury', 'Fatal Y/N', 'Time', 'Species ',
       'Source', 'pdf', 'href formula', 'href', 'Case Number', 'Case Number.1',
       'original order', 'Unnamed: 21', 'Unnamed: 22'],
      dtype='object')

In [128]:
# df.columns

In [202]:
df.rename(columns= {
    'Fatal Y/N': 'Fatal',
    'Species ': 'Species',
    'pdf': 'PDF', 
    'Sex': 'Gender',
    'href formula': 'Href_formula', 
    'href': 'Href',
    'Case Number': 'Case_Number', 
    'Case Number.1': 'Case_Number_1',
    'original order': 'Original_Order', 
    'Unnamed: 21': 'Unnamed_21', 
    'Unnamed: 22': 'Unnamed_22'
}, inplace=True)
print(df.columns)

Index(['Date', 'Year', 'Type', 'Country', 'State', 'Location', 'Activity',
       'Name', 'Gender', 'Age', 'Injury', 'Fatal', 'Time', 'Species', 'Source',
       'PDF', 'Href_formula', 'Href', 'Case_Number', 'Case_Number_1',
       'Original_Order', 'Unnamed_21', 'Unnamed_22'],
      dtype='object')


In [203]:
df.dtypes

Date               object
Year              float64
Type               object
Country            object
State              object
Location           object
Activity           object
Name               object
Gender             object
Age                object
Injury             object
Fatal              object
Time               object
Species            object
Source             object
PDF                object
Href_formula       object
Href               object
Case_Number        object
Case_Number_1      object
Original_Order    float64
Unnamed_21         object
Unnamed_22         object
dtype: object

In [204]:
# Here I am trying to identify the number of unique values for each column and determine which columns appear to be categorical. 
unique_counts = df.nunique() # Identify the number of unique values for each column
print("\nUnique value counts per column: ")
print(unique_counts)


Unique value counts per column: 
Date              6040
Year               261
Type                12
Country            239
State              920
Location          4553
Activity          1598
Name              5726
Gender               8
Age                246
Injury            4121
Fatal               12
Time               434
Species           1706
Source            5345
PDF               6789
Href_formula      6784
Href              6776
Case_Number       6777
Case_Number_1     6775
Original_Order    6797
Unnamed_21           1
Unnamed_22           2
dtype: int64


In [205]:
#Checking for null values in each column 
df.isna().any()


Date              False
Year               True
Type               True
Country            True
State              True
Location           True
Activity           True
Name               True
Gender             True
Age                True
Injury             True
Fatal              True
Time               True
Species            True
Source             True
PDF                True
Href_formula       True
Href               True
Case_Number        True
Case_Number_1      True
Original_Order     True
Unnamed_21         True
Unnamed_22         True
dtype: bool

In [187]:
df.isna().sum()

Date                 0
Year                 2
Type                18
Country             50
State              482
Location           565
Activity           585
Name               219
Gender             579
Age               2994
Injury              35
Fatal              561
Time              3526
Species           3132
Source              19
PDF                193
Href_formula       198
Href               196
Case_Number        194
Case_Number_1      195
Original_Order     193
Unnamed_21        6991
Unnamed_22        6990
dtype: int64

In [206]:
df.duplicated().sum()

0

In [207]:
df.duplicated().any()

False

In [190]:
df.describe()

,Year,Original_Order
count,6990.000000,6799.000000
mean,1935.018598,3401.152081
std,272.082794,1963.076319
min,0.000000,2.000000
25%,1947.000000,1701.500000
50%,1986.000000,3401.000000
75%,2009.000000,5100.500000
max,2026.000000,6802.000000


In [208]:
#Count the number of null values in each column
df.isna().sum()

Date                 0
Year                 2
Type                18
Country             50
State              482
Location           565
Activity           585
Name               219
Gender             579
Age               2994
Injury              35
Fatal              561
Time              3526
Species           3132
Source              19
PDF                193
Href_formula       198
Href               196
Case_Number        194
Case_Number_1      195
Original_Order     193
Unnamed_21        6991
Unnamed_22        6990
dtype: int64

In [209]:
df.Type.unique()

array(['Unprovoked', 'Provoked', ' Provoked', 'Questionable',
       'Watercraft', 'Sea Disaster', nan, '?', 'Unconfirmed',
       'Unverified', 'Invalid', 'Under investigation', 'Boat'],
      dtype=object)

In [210]:
print(df["Type"].value_counts()) 

Type
Unprovoked             5153
Provoked                637
Invalid                 552
Watercraft              355
Sea Disaster            242
Questionable             22
Boat                      7
 Provoked                 2
?                         1
Unconfirmed               1
Unverified                1
Under investigation       1
Name: count, dtype: int64


In [211]:
print(df["Type"].value_counts(dropna=False)) # I wanted to review the Nan 

Type
Unprovoked             5153
Provoked                637
Invalid                 552
Watercraft              355
Sea Disaster            242
Questionable             22
NaN                      18
Boat                      7
 Provoked                 2
?                         1
Unconfirmed               1
Unverified                1
Under investigation       1
Name: count, dtype: int64


In [212]:
#df_nan = df[df["Type"].isna()]
#df_nan

#3df["Type"] = df["Type"].isnull().sum

In [213]:
# Data Cleaning for the Column 'Type' 

#Reviewing how many were provoked and unprovoked 
print(df["Type"].unique())
# What is the meaning in here as "Provoked"?? 

['Unprovoked' 'Provoked' ' Provoked' 'Questionable' 'Watercraft'
 'Sea Disaster' nan '?' 'Unconfirmed' 'Unverified' 'Invalid'
 'Under investigation' 'Boat']


In [214]:
df["Type"] = df["Type"].str.strip()
df["Type"] = df["Type"].replace({
    " Provoked": "Provoked", 
    "Unconfirmed": "Unknown",
    "?" : "Unknown",
    "Invalid": "Unknown",
    "Under investigation": "Unknown",
    "Questionable": "Unknown", 
    "Watercraft" : "Water Vehicle", 
    "Boat" : "Water Vehicle"

})

df["Type"] = df["Type"].fillna("Unknown")
print(df["Type"].unique())

['Unprovoked' 'Provoked' 'Unknown' 'Water Vehicle' 'Sea Disaster'
 'Unverified']


In [215]:
print(df["Type"].unique())


['Unprovoked' 'Provoked' 'Unknown' 'Water Vehicle' 'Sea Disaster'
 'Unverified']


In [216]:
# Data Cleaning for the Column Gender
df["Gender"] = df["Gender"].replace({
    " M": "M", 
    "M ": "M", 
    "M x 2": "M", 
    " nan": "Unknown", 
    "lli": "Unknown", 
    "N": "M", 
    ".": "Unknown"
    })
df["Gender"] = df["Gender"].fillna("Unknown")
df.Gender.unique()

array(['M', 'F', 'Unknown'], dtype=object)

In [238]:
df["Species"].unique()
print(df_Species.unique()[:600])

['Unknown' 'Bronze whaler' 'Reportedly Tiger or Bull shark'
 'Great White Shart estimated 4.8m 16 ft' 'Reportedly a Tiger Shark'
 'Unknown ' '1m shark species unknown' 'Great White Shark'
 'Reported as a large Tiger shark' 'Bull or tiger 7-8ft' 'Bull shark'
 'Bull shark 6ft' 'Bull shark 6-7ft' 'Reportedly a Great White'
 'Reportedly Tiger Shark' 'Reef shark 6ft (1.8m)' '3m White Shark'
 'Undetermined small shall shark' '4-5ft Blacktip shark' 'Bull shark 6 ft'
 '6 ft shark' '6ft shark' 'Not stated area has Tiger and Galapogas sharks'
 'Not specified Bull shark highly suspect' 'Large shark' '2.5m shark'
 '3m Sandbar shark' 'Bull shark 3m ' 'Carribean reef shark 7ft'
 'Great White' 'Tiger shark' "Tiger shark 8'" "3' to 4' shark"
 "Bull shark, 7'" 'Blacktip reef shark' nan 'White shark'
 'Raggedtooth shark' "White shark, 13'" 'Broze whaler?' 'Sevengill shark'
 'Shark involment not confirmed' '1.8m bull shark' 'Lemon shark'
 'Oceanic whitetip shark' 'White Shark, 4.5 m' "8' tiger shark"
 "B

In [249]:
#Data Cleanining for the Column 'Species'
#df["Species"].unique() df["Species"] = df["Species"]
#df["Species"]
#df.Species.unique()

df_Species = df["Species"].fillna("Unknown").astype(str)

def clean_shark_name(species): 
    species = species.strip().lower()

    match = re.search(r"(great white|white|tiger|bull|hammerhead|blacktip|whale|nurse|mako|reef)s*shark", species)

    if match:
        return match.group(0).title()
    else: 
        return "Other/Unknown"

df_Species = df_Species.apply(clean_shark_name)

df_Species = df_Species.replace({
    "Not stated": "Unknown",
    "Not specified": "Unknown",
    "Invalid": "Unknown"
}) 

print(df_Species.value_counts())  # Check unique values
print(df_Species.tail())  # Display first few rows
#print(df_Species.nunique()) 
#print(df_Species.unique()[:100])

Species
Other/Unknown    6992
Name: count, dtype: int64
6987    Other/Unknown
6988    Other/Unknown
6989    Other/Unknown
6990    Other/Unknown
6991    Other/Unknown
Name: Species, dtype: object


In [246]:
df_Species
#print(df.Species)

0                                      Unknown
1                                Bronze whaler
2                                      Unknown
3               Reportedly Tiger or Bull shark
4       Great White Shart estimated 4.8m 16 ft
                         ...                  
6987                                       NaN
6988                                       NaN
6989                                       NaN
6990                                       NaN
6991                                       NaN
Name: Species, Length: 6992, dtype: object

In [ ]:
df_Species.fillna("Unknown") # Filling every 0's with unknown 


df_Species

0                                      Unknown
1                                Bronze whaler
2                                      Unknown
3               Reportedly Tiger or Bull shark
4       Great White Shart estimated 4.8m 16 ft
                         ...                  
6987                                       NaN
6988                                       NaN
6989                                       NaN
6990                                       NaN
6991                                       NaN
Name: Species, Length: 6992, dtype: object

In [ ]:
df["Species"] = df["Species"].str.extract(r'([A-Za-z\s-]+)').fillna("Unknown") # Cleaning here the unnecessary details
df

In [ ]:
df["Species"] = df["Species"]

In [ ]:
#df["Date"] = pd.to_datetime(df["Date"], errors="coerce")

In [155]:
# Data Cleaning for the column 'Country', I've tried to use regex 
df["Country"] = df["Country"].str.strip().str.title()
df["Country"] = df["Country"].str.replace(r"[^a-zA-Z\s]", "", regex=True)
#df["Country"] = df["Country"].map(lambda x: x.upper())
df["Country"] = df["Country"].fillna("Unknown")
print(df["Country"].unique())

['Australia' 'Usa' 'New Caledonia' 'Egypt' 'Thailand' 'New Zealand'
 'Maldives' 'Hawaii' 'Honduras' 'Morocco' 'Jamaica' 'Belize'
 'Maldive Islands' 'Turks And Caicos' 'French Polynesia' 'Tobago'
 'Bahamas' 'India' 'Trinidad' 'South Africa' 'Mexico' 'Philippines'
 'Coral Sea' 'Spain' 'Portugal' 'Samoa' 'Colombia' 'Ecuador' 'Cuba'
 'Brazil' 'Seychelles' 'Argentina' 'Fiji' 'England' 'Japan' 'Indonesia'
 'Columbia' 'Costa Rica' 'British Overseas Territory' 'Canada' 'Jordan'
 'St Kitts  Nevis' 'St Martin' 'Papua New Guinea' 'Reunion Island'
 'Israel' 'China' 'Ireland' 'Italy' 'Malaysia' 'Libya' 'Unknown'
 'Mauritius' 'Solomon Islands' 'St Helena British Overseas Territory'
 'Comoros' 'Reunion' 'United Kingdom' 'United Arab Emirates' 'Cape Verde'
 'Dominican Republic' 'Cayman Islands' 'Aruba' 'Mozambique' 'Puerto Rico'
 'Atlantic Ocean' 'Greece' 'France' 'Trinidad  Tobago' 'Kiribati'
 'Diego Garcia' 'Taiwan' 'Palestinian Territories' 'Guam' 'Nigeria'
 'Tonga' 'Scotland' 'Croatia' 'Saudi Arab

In [ ]:
# Cleaning for location
df["Location"] = df["Location"].astype(str)
df["Location"] = df["Location"].map(lambda x:x.upper())
df["Location"] = df["Location"].fillna("Unknown")
df["Location"] = df["Location"].str.strip()
df["Location"] = df["Location"].replace({
    "NAN": "Unknown",
     "nan": "Unknown",
      "Nan": "Unknown", 
})

In [ ]:
# Data Cleaning for the Column 'State'
#df["State"] = df["State"].
df["State"] = df["State"].fillna("Unknown")
df["State"] = df["State"].astype(str)
df["State"] = df["State"].str.strip()
df["State"] = df["State"].replace({
    "NAN": "Unknown",
    "nan": "Unknown",
    "Nan": "Unknown", 
    " ": "Unknown",
    "  ": "Unknown"
})
print(df["State"].unique())

In [ ]:
# Data Cleaning for the Column Activity 
df["Activity"] = df["Activity"].fillna("Unknown")
df["Activity"] = df["Activity"].astype(str)
df["Activity"] = df["Activity"].str.strip()
df["Activity"] = df["Activity"].replace({
    "NAN": "Unknown",
     "nan": "Unknown",
      "Nan": "Unknown",
})
# Needs more cleaning 

In [ ]:
df.isna().sum()

In [ ]:
# Data Cleaning for the Column 'Name' 
df["Activity"] = df["Activity"].fillna("Unknown")
df["Activity"] = df["Activity"].astype(str)
df["Activity"] = df["Activity"].str.strip()
df["Activity"] = df["Activity"].replace({
    "NAN": "Unknown",
     "nan": "Unknown",
      "Nan": "Unknown",
})
# Needs to be furthered cleaned 

In [ ]:
# Data Cleaning for the Column Age
# df["Age"] = df["Age"].astype()
# df["Age"] = df["Age"]
# df["Age"] = df["Age"]


In [ ]:
# Data Cleaning for The Column Fatal
df["Fatal"] = df["Fatal"].fillna("Unknown")
df["Fatal"] = df["Fatal"].astype(str)
df["Fatal"] = df["Fatal"].replace({
    "Nan": "Unknown",
    " N": "N",
    "UNKNOWN": "Unknown",
    "F": "Unknown",\
    "M": "N",
    "n": "N",
    "Nq": "N",
     2017: "Unknown",
     "Y x 2": "Y",
     "y": "Y",
     "N   ": "N",
})
#df["Fatal"] = df["Fatal"]
#df["Fatal"] = df["Fatal"]
#df["Fatal"] = df["Fatal"]

In [ ]:
df['Fatal'].value_counts(dropna=False)

In [ ]:
df_count = df.groupby("Year")["Date"].count()
df_count

In [ ]:
df_count = df.groupby("Year")["Date"].count()
df_count[df_count.index > 1750].plot()

In [ ]:
df_count#.isna().sum()

In [ ]:
df[df["Year"] == 5]

In [ ]:
df["Country"].value_counts()


In [ ]:
df_usa = df[df["Country"] == "USA"]
df_usa

In [ ]:
df_usa["Date"] = pd.to_datetime(df_usa["Date"])

In [ ]:
df_usa["Date"].value_counts()

In [ ]:
pd.to_datetime(df_usa["Data_Clean"])